In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pyspark.sql.functions as F
import setuptools
import statsmodels.api as sm
from dotenv import load_dotenv
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from sklearn import linear_model, preprocessing
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
%matplotlib inline

load_dotenv()

In [ ]:
df = pd.read_parquet('data/preprocessed_regression_data')
df.head()

In [ ]:
print(df.shape[0])
df = df.dropna()
print(df.shape[0])

In [ ]:
y = df['electric']
X = df[['vtax', 'price', 'population', 'fuelCost', 'cents_kWh', 'hh_med_income']]

pipe = Pipeline([
    ('scaler', StandardScaler()), # zero mean unit variance
    #('polyd2', PolynomialFeatures(degree=2))
])

Xp = pipe.fit_transform(X)
#Xp = pd.DataFrame(Xp, columns=pipe.named_steps['polyd2'].get_feature_names_out())
#Xp.head()

fit = pipe.fit(Xp, y)

smfit = sm.Logit(y, sm.add_constant(Xp)).fit()
smfit.summary()

In [ ]:
smfit.pvalues

In [ ]:
pipe.named_steps['polyd2'].get_feature_names_out()